# STEP 2
## Data Exploration
## Featurization

In [46]:
import sqlite3
import csv
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Crippen
import numpy as np

In [47]:
conn = sqlite3.connect('sabs_moonshot.db')

# Extract the compounds with missing descriptors

In [48]:
df = pd.read_sql_query("""
SELECT 
    compounds.id,
    compounds.MW,
    rdkit_descriptors.HBA,
    rdkit_descriptors.HBD, rdkit_descriptors.cLogP,
    compounds.smiles
FROM rdkit_descriptors 
INNER JOIN compounds on compounds.id = rdkit_descriptors.compound_id
WHERE HBA = "" OR HBD = "" OR MW = "" OR cLogP = "" AND assayed = TRUE;
""",
conn)
print(df)

                     id MW HBA HBD cLogP  \
0    DAR-DIA-6a508060-1                    
1    DAR-DIA-6a508060-2                    
2    DAR-DIA-6a508060-3                    
3    DAR-DIA-6a508060-4                    
4    DAR-DIA-6a508060-5                    
5    DAR-DIA-6a508060-7                    
6    DAR-DIA-6a508060-8                    
7    DAR-DIA-6a508060-9                    
8   DAR-DIA-6a508060-10                    
9   DAR-DIA-6a508060-11                    
10  DAR-DIA-6a508060-12                    
11  DAR-DIA-6a508060-13                    
12  DAR-DIA-6a508060-14                    
13  DAR-DIA-6a508060-15                    
14  DAR-DIA-6a508060-16                    
15   ALP-UNI-b33a865d-1                    
16   ALP-UNI-b33a865d-1                    
17   ALP-UNI-b33a865d-1                    
18   DAR-DIA-6a508060-3                    
19   DAR-DIA-6a508060-4                    
20   PRA-UNK-2c426785-1                    
21   EDG-MED-0c930815-1         

# Using RDKit to compute molecular descriptors and Lipinski's rule violations

In [51]:
class Molecule:
    """ A molecule. In particular, either the R1 or R2 group, or the scaffold
    and one or two groups.
    There are methods which tell you the properties of the molecule and if it
    passes the Lipsinki test
    """

    def __init__(self, mol_smiles):
        """Constructor for Molecule class. Initialises Molecule instance from
        smile string.
        :param mol_smiles: smile string of molecule
        :type mol_smiles: String
        """
        self.__mol_smiles = mol_smiles

    # @property
    # def get_smile_string(self):
    #     """Returns molecule's smile string
    #     :return: smile string of molecule
    #     :rtype: String
    #     """
    #     return self.__mol_smiles

    def descriptors(self):
        """Calculate molecule descriptor metrics as dict:
        | mol - smile string
        | MW - molecular weight
        | logP - logP
        | TPSA - topological polar surface area
        | HA - heavy atom count
        | h_acc - H acceptor count
        | h_don - H donator count
        | rings - ring count
        :return: molecule descriptor metrics
        :rtype: dict
        """
        mol = Chem.MolFromSmiles(self.__mol_smiles)
        mw = Descriptors.ExactMolWt(mol)
        log_p = Crippen.MolLogP(mol)
        tpsa = rdMolDescriptors.CalcTPSA(mol)  # topological polar surface area
        ha = Lipinski.HeavyAtomCount(mol)  # heavy atom count
        h_acceptors = Lipinski.NumHAcceptors(mol)
        h_donors = Lipinski.NumHDonors(mol)
        rings = Lipinski.RingCount(mol)
        molar_refractivity=Chem.Crippen.MolMR(mol)
        rotatable_bonds= Descriptors.NumRotatableBonds(mol)
        desc_dict = {'smiles': self.__mol_smiles, #changed from mol to smiles so consistent with database we downloaded
                     'MW': mw,
                     'logP': log_p,
                     'TPSA': tpsa,
                     'HA': ha,
                     'h_acc': h_acceptors,
                     'h_don': h_donors,
                     'rings': rings,
                     'MR': molar_refractivity, 
                     'Rot_bonds': rotatable_bonds
                     }
        return desc_dict

    def lipinski(self, desc_dict):
        """Calculate Lipinski from the descriptor dictionary. Returns the
        number of rules broken and whether the molecule passes.
        :param desc_dict: molecule descriptor metrics
        :type desc_dict: dict
        :return: violations
        :rtype: dict
        """
        violations = {'MW': desc_dict['MW'] < 500.0,
                      'h_acc': desc_dict['h_acc'] <= 10,
                      'h_don': desc_dict['h_don'] <= 5,
                      'logP': desc_dict['logP'] < 5}
        return violations


Select smiles strings from database

In [52]:
df = pd.read_sql_query("""
SELECT compounds.id, compounds.smiles
FROM compounds 
INNER JOIN assays ON compounds.id = assays.compound_id
WHERE assays.r_avg_IC50 != "" AND assays.f_avg_IC50 != "" ;
""",
conn)
df

,id,smiles
0,DAR-DIA-23aa0b97-19,N#Cc1cccc(NC(=O)Cc2cncc3ccccc23)c1
1,DAR-DIA-23aa0b97-20,O=C(Cc1cncc2ccccc12)Nc1ccccc1
2,TRY-UNI-714a760b-3,Cc1c(N)cncc1NC(=O)Cc1cccc(Cl)c1
3,TRY-UNI-714a760b-6,Cc1ccncc1NC(=O)Cc1cccc(Cl)c1
4,TRY-UNI-714a760b-12,Cc1ccncc1NC(=O)Nc1cccc(Cl)c1
...,...,...
656,WAR-XCH-79d12f6e-6,Cc1ccc(C)c(S(=O)(=O)N2CCN(C(=O)CCl)CC2)c1
657,ALP-POS-869ac754-1,O=C(Nc1cncc2ccccc12)C1CCOc2cc(Cl)c(Cl)cc21
658,MED-COV-4280ac29-15,O=C(CCl)N1CCN(Cc2cccc(Cl)c2)CC1
659,LON-WEI-8f408cad-4,O=C(CCl)N1CCN(S(=O)(=O)c2cccc(F)c2)CC1


# Converting compounds database to pandas dataframe

In [8]:
# df = pd.read_sql_query("""
# SELECT *
# FROM compounds;
# """,
# conn)
# print(df[:10])

                   id                                             smiles  \
0  ANT-DIA-3c79be55-1                 N#Cc1ccccc1NC(=O)Cc1c[nH]c2ncccc12   
1  ANT-DIA-3c79be55-2                         N#Cc1ccccc1NC(=O)Cc1cccnc1   
2  ANT-DIA-3c79be55-3          CCNc1ccc(C#N)c(NC(=O)Cc2c[nH]c3ncccc23)c1   
3  ANT-DIA-3c79be55-4                 CS(=O)(=O)Cc1ccc(C(=O)Nc2cccnc2)o1   
4  ANT-DIA-3c79be55-5                 O=C(Nc1cccnc1)c1ccc(N2CCC(O)CC2)o1   
5  ROB-UNI-b2e39629-1                      CCNc1ccc(C#N)cc1CCNS(C)(=O)=O   
6  ROB-UNI-b2e39629-2   CS(=O)(=O)NCCc1c[nH]c2c(CCNS(C)(=O)=O)cc(Cl)cc12   
7  ROB-UNI-b2e39629-3  CCn1cc(CCNS(C)(=O)=O)c2cc(C#N)cc(CCNS(C)(=O)=O...   
8  ROB-UNI-b2e39629-4       CC(=O)NCCc1c[nH]c2c(CCNS(C)(=O)=O)cc(Cl)cc12   
9  ROB-UNI-b2e39629-5     CCn1cc(CCNC(C)=O)c2cc(C#N)cc(CCNS(C)(=O)=O)c21   

        MW NMR_std_ratio  assayed  
0  276.299                      0  
1  237.262                      0  
2  319.368                      0  
3  280.305         

In [9]:

# new_df=pd.DataFrame()
# descriptors={'a': 65, 'b': 66}
# new_df=new_df.append(descriptors, ignore_index=True)
# print(new_df)

In [10]:
# for i in range(len(df)):
#     cur_smile=df.iloc[i]['smiles']
#     print(cur_smile)
#     cur_mol=Molecule(cur_smile)
#     cur_mol.descriptors

# Creating dataframe with smiles and molecular descriptors

In [53]:
# new_df=pd.DataFrame()
desc_dict = [
                     'MW' ,
                     'logP' ,
                     'TPSA' ,
                     'HA' ,
                     'h_acc',
                     'h_don',
                     'rings',
                     'MR',
                     'Rot_bonds'
]

for desc in desc_dict:
    df[desc]=np.nan
df["lipinski"]=np.nan

for i in range(len(df)): #change to range(len(df))
    cur_smile=df.iloc[i]['smiles']
    # print(cur_smile)
    cur_mol=Molecule(cur_smile)
    descriptors=cur_mol.descriptors()
    lipinski=cur_mol.lipinski(descriptors)
    # for 
    false_num= 4 - sum(lipinski.values()) # no. falses= 4 - no. trues
    df["lipinski"].iloc[i]=false_num
    # print(descriptors)
    for key in desc_dict:
        df[key].iloc[i]=descriptors[key]
    

    # df["test"].iloc[i]= 2
    


df.to_csv('./compounds-featurized.csv')
df

/usr/local/anaconda3.7/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,id,smiles,MW,logP,TPSA,HA,h_acc,h_don,rings,MR,Rot_bonds,lipinski
0,DAR-DIA-23aa0b97-19,N#Cc1cccc(NC(=O)Cc2cncc3ccccc23)c1,287.105862,3.28768,65.78,22.0,3.0,1.0,3.0,85.2337,3.0,0.0
1,DAR-DIA-23aa0b97-20,O=C(Cc1cncc2ccccc12)Nc1ccccc1,262.110613,3.41600,41.99,20.0,2.0,1.0,3.0,80.5187,3.0,0.0
2,TRY-UNI-714a760b-3,Cc1c(N)cncc1NC(=O)Cc1cccc(Cl)c1,275.082540,2.80682,68.01,19.0,3.0,2.0,2.0,77.1721,3.0,0.0
3,TRY-UNI-714a760b-6,Cc1ccncc1NC(=O)Cc1cccc(Cl)c1,260.071641,3.22462,41.99,18.0,2.0,1.0,2.0,72.7597,3.0,0.0
4,TRY-UNI-714a760b-12,Cc1ccncc1NC(=O)Nc1cccc(Cl)c1,261.066890,3.68742,54.02,18.0,2.0,2.0,2.0,73.0674,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
656,WAR-XCH-79d12f6e-6,Cc1ccc(C)c(S(=O)(=O)N2CCN(C(=O)CCl)CC2)c1,330.080491,1.37514,57.69,21.0,3.0,0.0,2.0,81.9748,3.0,0.0
657,ALP-POS-869ac754-1,O=C(Nc1cncc2ccccc12)C1CCOc2cc(Cl)c(Cl)cc21,372.043233,5.04640,51.22,25.0,3.0,1.0,4.0,99.6877,2.0,1.0
658,MED-COV-4280ac29-15,O=C(CCl)N1CCN(Cc2cccc(Cl)c2)CC1,286.063968,2.22300,23.55,18.0,2.0,0.0,2.0,74.0290,3.0,0.0
659,LON-WEI-8f408cad-4,O=C(CCl)N1CCN(S(=O)(=O)c2cccc(F)c2)CC1,320.039769,0.89740,57.69,20.0,3.0,0.0,2.0,72.4588,3.0,0.0


# Creating dataframe with molecular descriptors and finding the Lipinski's rule of five violations for each compound

In [12]:
new_df=pd.DataFrame()
for i in range(2): #change to range(len(df))
    cur_smile=df.iloc[i]['smiles']
    cur_mol=Molecule(cur_smile)
    descriptors=cur_mol.descriptors()
    lipinski=cur_mol.lipinski(descriptors)
    print(lipinski)
    new_df=new_df.append(descriptors, ignore_index=True)
    
    new_df=new_df.append(lipinski, ignore_index=True)

print(new_df)

{'MW': True, 'h_acc': True, 'h_don': True, 'logP': True}
{'MW': True, 'h_acc': True, 'h_don': True, 'logP': True}
     HA          MW   TPSA  h_acc  h_don     logP  rings  \
0  21.0  276.101111  81.57    3.0    2.0  2.61578    3.0   
1   NaN    1.000000    NaN    1.0    1.0  1.00000    NaN   
2  18.0  237.090212  65.78    3.0    1.0  2.13448    2.0   
3   NaN    1.000000    NaN    1.0    1.0  1.00000    NaN   

                               smiles  
0  N#Cc1ccccc1NC(=O)Cc1c[nH]c2ncccc12  
1                                 NaN  
2          N#Cc1ccccc1NC(=O)Cc1cccnc1  
3                                 NaN  


# Trying to discard compounds that fail Lipinski's rule of 5

In [13]:
lipinski_list = ['MW', 'logP', 'h_acc', 'h_don']
    # r_group_1_id = request.args.get('r1')
    # r_group_2_id = request.args.get('r2')
    # molecule_key = tuple2str((r_group_1_id, r_group_2_id))
    # drug_mol = FinalMolecule(r_group_1_id, r_group_2_id)
    # drug_lipinski = drug_mol.lipinski(drug_mol.descriptors())
    # lipinski_dict = {cur_mol: lipinski}
for label in lipinski_list:
    if "False" not in lipinski:
        pass
    # else:
    #     # delete smiles/compound row

# Trying to discard compounds that violate more than one of Lipinski's rule of 5

In [14]:

# for label in lipinski_list:
#     if "False" appears twice  in lipinski:
#         #delete it
#     # else:
#         pass

# Variants of Lipinski's rule of 5

# Molar refractivity from 40 to 130, 10 or fewer rotatable bonds, Polar surface area no greater than 140 Å2. Drugs are predicted to have predicted to have good oral bioavailability when the last 2 of these are satisfied.

# TPSA is calculated above

# Not calculating these descriptors - not sure why

In [35]:
mol1=Molecule("OCC")
descriptors=mol1.descriptors()
lipinski=mol1.lipinski(descriptors)
print(lipinski)
h_acceptors = Lipinski.NumHAcceptors(mol1)
print(h_acceptors)
molar_refractivity=Chem.Crippen.MolMR(mol1)
rotatable_bonds= Descriptors.NumRotatableBonds(mol1)
print(molar_refractivity)
print(rotatable_bonds)


{'MW': True, 'h_acc': True, 'h_don': True, 'logP': True}


ArgumentError: Python argument types in
    rdkit.Chem.rdMolDescriptors.CalcNumHBA(Molecule)
did not match C++ signature:
    CalcNumHBA(RDKit::ROMol mol)